Open a new terminal in this Jupyter Lab to automatically log into Azure at startup. Using the terminal,

* `bash` activate the bash shell.

In [1]:
import pandas as pd

In [2]:
from azure.identity import AzureCliCredential
credential = AzureCliCredential()
token = credential.get_token("https://westeurope.asazure.windows.net").token

In [3]:
from clr_loader import get_coreclr
from pythonnet import set_runtime
rt = get_coreclr("/root/runtimeconfig.json")
set_runtime(rt)

In [4]:
import clr
clr.AddReference('/root/dll/Microsoft.AnalysisServices.AdomdClient.dll')
from Microsoft.AnalysisServices.AdomdClient import AdomdConnection, AdomdCommand, AdomdDataReader

In [5]:
conn = AdomdConnection(f'Data Source=asazure://westeurope.asazure.windows.net/unhcrfinbiprod;Password={token}')
conn.Open()

In [6]:
commandText = '''
EVALUATE
SUMMARIZE(
    FILTER('Commitment Control Ledger', 'Commitment Control Ledger'[Budget Year] in { "2004", "2005", "2006" }),
    'Commitment Control Ledger'[Budget Year],
    "TOTAL",
    [KK Activity Base Amount]
)
'''
cmd = AdomdCommand(commandText, conn)
reader = cmd.ExecuteReader()
cols = [row['ColumnName'] for row in reader.GetSchemaTable().Rows]
df = pd.DataFrame(columns=cols)
while reader.Read():
    values = [[reader[i]] for i in range(len(cols))]
    row = dict(zip(cols, values))
    df_row = pd.DataFrame.from_dict(row)
    df = pd.concat([df, df_row], ignore_index=True)
df

,Commitment Control Ledger[Budget Year],[TOTAL]
0,2006,-10161.83
1,2004,-4797.77
2,2005,-2395.62


In [7]:
reader.Close()

In [8]:
conn.Close()